In [ ]:
# -*- coding:utf-8 -*-

from data_tools.api import *
from utilscht.Data import *
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from joblib import Parallel,delayed
import tushare as ts
import math
import datetime
import sys
import logging


##命令行获得日期参数
if 0:
    TODAY_DATE=str(sys.argv[1])
    NEXT_DATE=str(get_next_trade_date(TODAY_DATE))
    month = int(TODAY_DATE[4:6])
    if month >= 5:
        START_YEAR=int(TODAY_DATE[0:4])-1 #前一年作为基准年
    else:
        START_YEAR=int(TODAY_DATE[0:4])-2 #前前一年作为基准年
#当日日期作为日期参数
else:
    NEXT_DATE=str(datetime.datetime.now().date())
    NEXT_DATE=NEXT_DATE.replace('-','')
    TODAY_DATE=str(get_previous_trade_date(NEXT_DATE))
    month = int(TODAY_DATE[4:6])
    if month >= 5:
        START_YEAR=int(TODAY_DATE[0:4])-1 #前一年作为基准年
    else:
        START_YEAR=int(TODAY_DATE[0:4])-2 #前前一年作为基准年

DB_INFO = dict(host='192.168.1.234',
               user='winduser',
               password='1qaz@WSX',
               db='wind')

conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)


logging.basicConfig(level=logging.DEBUG,#控制台打印的日志级别
                    filename=r'/home/ywang/proj_con_expect/logging/con_expect_strategy.log',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format='%(asctime)s - %(pathname)s[line:%(lineno)d] - %(levelname)s: %(message)s'#日志格式
                   )


def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=True, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)





###获得所需数据
def Get_Required_Data():
    #获取行业和市值
    indus_size_df=query_table("DailyBar",start_date="{}0101".format(START_YEAR),end_date=TODAY_DATE,fields=["L1_INDUSTRY","mktcap"])
    indus_size_df["DataDate"]=indus_size_df["DataDate"].apply(lambda x:str(x))
    print("finish")

    #获取研报预期数据
    sql = """SELECT S_INFO_WINDCODE, EST_DT, REPORTING_PERIOD,RESEARCH_INST_NAME, ANALYST_NAME,EST_NET_PROFIT from ASHAREEARNINGEST where EST_DT between '{}0101' and '20201231'""".format(START_YEAR)   
    df_est = pd.read_sql_query(sql, conn)
    df_est.rename(columns={"S_INFO_WINDCODE":"sid","EST_DT":"DataDate"},inplace=True)
    df_est=df_est.sort_values(["sid","DataDate","REPORTING_PERIOD"])
    df_est=pd.merge(df_est,indus_size_df,on=["sid","DataDate"],how="left")

    #获取发布的年报数据
    sql = """SELECT S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD,S_FA_DEDUCTEDPROFIT,S_FA_EXTRAORDINARY
    from ASHAREFINANCIALINDICATOR where ANN_DT between '{}0101' and '20201231'""".format(START_YEAR)
    df_FA_EPS = pd.read_sql_query(sql, conn)
    df_FA_EPS.rename(columns={"S_INFO_WINDCODE":"sid","REPORT_PERIOD":"REPORTING_PERIOD"},inplace=True)
    df_est=pd.merge(df_est,df_FA_EPS,on=["sid","REPORTING_PERIOD"],how="left")

    #获取总股本信息
    sql="select S_INFO_WINDCODE,TOT_SHR,CHANGE_DT from AShareCapitalization".upper()
    df_tot_share=pd.read_sql_query(sql,conn)
    df_tot_share.rename(columns={"S_INFO_WINDCODE":"sid"},inplace=True)
    df_tot_share=df_tot_share.sort_values(["sid","CHANGE_DT"])
    df_now_tot_share=df_tot_share.groupby('sid',as_index=False).apply(lambda x:x.iloc[-1])

    #计算前复权每股预期收益
    df_est=pd.merge(df_est,df_now_tot_share,on='sid',how='left')
    df_est['EST_EPS_DILUTED']=df_est['EST_NET_PROFIT']/df_est['TOT_SHR']
    df_est['S_FA_EPS_DILUTED']=(df_est['S_FA_DEDUCTEDPROFIT']+df_est['S_FA_EXTRAORDINARY'])/df_est['TOT_SHR']/10000

    #取企业快报数据
    sql_1 = """SELECT S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD,NET_PROFIT_EXCL_MIN_INT_INC
    from ASHAREPROFITEXPRESS where ANN_DT between '{}0101' and '20201231'""".format(START_YEAR)

    df_ProfitExpress = pd.read_sql_query(sql_1, conn)
    df_ProfitExpress.rename(columns={"S_INFO_WINDCODE":"sid","REPORT_PERIOD":"REPORTING_PERIOD"},inplace=True)
    print("finish")

    #取企业预报数据
    sql_2 = """SELECT S_INFO_WINDCODE, S_PROFITNOTICE_DATE, S_PROFITNOTICE_PERIOD,S_PROFITNOTICE_NETPROFITMIN,S_PROFITNOTICE_NETPROFITMAX
     from ASHAREPROFITNOTICE where S_PROFITNOTICE_DATE between '{}0101' and '20201231'""".format(START_YEAR)
    df_ProfitNotice = pd.read_sql_query(sql_2, conn)
    df_ProfitNotice.rename(columns={"S_INFO_WINDCODE":"sid","S_PROFITNOTICE_PERIOD":"REPORTING_PERIOD"},inplace=True)
    print("finish")

    #获得每天的收盘价序列

    df_close=query_table("DailyBar",start_date="{}0101".format(START_YEAR),end_date=TODAY_DATE,fields=["close"])
    df_close=df_close.rename(columns={"S_INFO_WINDCODE":"sid","TRADE_DT":"DataDate"})
    df_close=df_close.sort_values(["sid","DataDate"])
    df_close["DataDate"]=df_close["DataDate"].apply(str)
    #df_close.to_csv(r"data/close_price.csv",index=False)
    print("finish")


    #获得过去两年PE_TTM的 rolling_median 值（相较于mean值更为稳定，不易受极端值影响）

    df_PETTM=query_table("DailyBar",start_date="{}0101".format(START_YEAR-1),\
                         end_date=TODAY_DATE,fields=["pe_ttm","L1_INDUSTRY"])
    df_PETTM=df_PETTM.rename(columns={"S_INFO_WINDCODE":"sid","TRADE_DT":"DataDate","pe_ttm":"S_VAL_PE_TTM"})
    df_PETTM=df_PETTM.sort_values(["sid","DataDate"])
    df_PETTM["DataDate"]=df_PETTM["DataDate"].apply(str)

    def adj_by_indus_pe(df):
        indus_pe = np.nanmedian(df["S_VAL_PE_TTM"]) 
        stk_pe= df["S_VAL_PE_TTM"].values
        df["S_VAL_PE_TTM"] = np.clip(stk_pe, 0,indus_pe*2)
        return df
    groups = df_PETTM.groupby(["L1_INDUSTRY","DataDate"])
    df_PETTM = apply_parallel(groups, adj_by_indus_pe).reset_index(drop=True)
    df_PETTM = df_PETTM.sort_values(["sid","DataDate"])

    def Rolling_Median(df):
        df["S_VAL_PE_TTM"]=df["S_VAL_PE_TTM"].fillna(method='ffill')
        df["pe_2y_rollingmedian"]=df["S_VAL_PE_TTM"].rolling(252*3).apply(lambda x:np.percentile(x,40))
        return df

    grouped=df_PETTM.groupby("sid")
    df_PETTM_2y_rollingmedian=apply_parallel(grouped,Rolling_Median)
    print("finish")

    return  df_est,df_now_tot_share,df_FA_EPS,df_ProfitExpress,df_ProfitNotice,df_close,df_PETTM_2y_rollingmedian

df_est,df_now_tot_share,df_FA_EPS,df_ProfitExpress,df_ProfitNotice,df_close,df_PETTM_2y_rollingmedian=Get_Required_Data()
print("finish getting data\n\n")

###利用去年一致预期数据和今年实际年报数据获得分析师偏差
def Get_Analyst_Bias(df_est):
    #计算距离年末的月数（第二年初的预测月数为负值）
    df_1y_est=df_est.groupby(["sid","DataDate","ANALYST_NAME"],as_index=False).apply(lambda x:x.iloc[0]).reset_index(drop=True)
    df_1y_est["Date_to_Reporting"]=df_1y_est[["DataDate","REPORTING_PERIOD"]].apply(lambda x:int(x[1][4:6])-int(x[0][4:6]) if x[0][0:4]==x[1][0:4] else int(x[1][4:6])-int(x[0][4:6])-12,axis=1)

    #计算偏差比例
    df_1y_est["Analyst_Bias"]=np.abs(df_1y_est["S_FA_EPS_DILUTED"]-df_1y_est["EST_EPS_DILUTED"])/df_1y_est["S_FA_EPS_DILUTED"]

    #按照年度将偏差对行业、市值、距离年末月数进行回归取残差
    def Regress_by_Year(df):
        df=df[pd.notnull(df["Analyst_Bias"])&pd.notnull(df["L1_INDUSTRY"])]
        if len(df)==0:
            return df
        bias=np.array(df["Analyst_Bias"])

        indus_dummy=np.array(pd.get_dummies(df["L1_INDUSTRY"]))
        mktv=np.array(df["mktcap"])
        date_to_reporting=np.array(df["Date_to_Reporting"])

        model=sm.OLS(bias,np.column_stack([indus_dummy,mktv,date_to_reporting]))
        result=model.fit()
        df["adj_analyst_bias"]=result.resid

        return df

    df_1y_est["adj_analyst_bias"]=np.nan
    df_1y_est=df_1y_est.groupby("REPORTING_PERIOD",as_index=False).apply(lambda x:Regress_by_Year(x))
    
    #按照预测年度、分析师来计算偏差平均值作为下一年度该分析师的可信水平
    analyst_avg_bias=df_1y_est.groupby(["REPORTING_PERIOD","ANALYST_NAME"],as_index=False).apply(lambda x:np.nan if len(x["adj_analyst_bias"])<5 else np.mean(x["adj_analyst_bias"]))
    analyst_avg_bias=analyst_avg_bias.to_frame(name="analyst_avg_bias").reset_index()
    analyst_avg_bias["next_reporting_period"]=analyst_avg_bias["REPORTING_PERIOD"].apply(lambda x:str(int(x)+10000))
    del analyst_avg_bias["REPORTING_PERIOD"]

    df_est["ANN_PERIOD"]=df_est["DataDate"].apply(lambda x:x[0:4]+"1231" if x[4:6]>'04' else str(int(x[0:4])-1)+'1231')
    df_est=pd.merge(df_est,analyst_avg_bias,left_on=["ANALYST_NAME","ANN_PERIOD"],right_on=["ANALYST_NAME","next_reporting_period"],how="left")
    print("finish")

    return df_est
df_est=Get_Analyst_Bias(df_est)
print("finish getting bias\n\n")


###对过去3个月（过去三个月研报数小于10的个股使用过去6月的数据，对于过去六个月没有研报的个股填充nan值）的数据进行过滤，同一分析师对同一个股的多篇研报只取最后一篇
###并对过去三个月的一致预期数据按照时间和分析师偏差进行双重加权（时间越近权重越大，分析师偏差越小权重越大）
def Get_Con_Est_Eps(df_est):
    date_calendar=pd.date_range("{}0101".format(START_YEAR),TODAY_DATE)
    date_calendar=[str(i)[0:4]+str(i)[5:7]+str(i)[8:10] for i in date_calendar]

    #按照分析师上一年预测偏差以及研报发布时间进行双重加权
    def con_est_calcu(est_data):
        if len(est_data) == 0:
            return np.nan
        
        
        analyst_bias=est_data["analyst_avg_bias"]
        max_bias=np.nanmax(analyst_bias)
        min_bias=np.nanmin(analyst_bias)
        if max_bias==min_bias or (math.isnan(min_bias) or math.isnan(max_bias)):
            num = len(analyst_bias)
            weight_analyst=np.array([1/num]*num)
        else:    
            weight_analyst=analyst_bias.apply(lambda x:1 if math.isnan(x) else 3-2*(x-min_bias)/(max_bias-min_bias))
        weight_analyst=weight_analyst/np.sum(weight_analyst)

        est_data["date_to_now"]=est_data["DataDate"].apply(lambda x:date_calendar.index(TODAY_DATE)-date_calendar.index(x))
        weight_date=np.power(0.5**(1/45),np.array(est_data["date_to_now"]))
        weight_date=weight_date/np.sum(weight_date)

        total_weight=weight_analyst*weight_date
        total_weight=np.array(total_weight/np.sum(total_weight))

        est_eps=est_data["EST_EPS_DILUTED"].values
        median = np.median(est_eps)
        MAD =  1.483 * np.median(np.abs(est_eps - median))
        est_eps = np.clip(est_eps,median-3*MAD,median+3*MAD)
        
        if len(est_eps)>=3:
            max_index = np.argmax(est_eps)
            min_index = np.argmin(est_eps)
            est_eps = np.delete(est_eps,[max_index,min_index])
            total_weight = np.delete(total_weight,[max_index,min_index])
            total_weight = total_weight/np.sum(total_weight)
        con_est_eps=np.sum(np.array(est_eps)*total_weight)

        return con_est_eps

    def Process_Single_StkPeriod(df):

        date_range=date_calendar[-90:]#取过去90天数据（包含今天）
        est_data=df[df["DataDate"].isin(date_range)]
        est_data=est_data[pd.notnull(est_data["EST_EPS_DILUTED"])]
        est_data=est_data.groupby("ANALYST_NAME").apply(lambda x:x.iloc[-1])#同一分析师多篇研报只取最后一篇

        #如果过去90天研报数小于10，则取过去180天研报
        if len(est_data)<10:
            date_range=date_calendar[-180:]
            est_data=df[df["DataDate"].isin(date_range)]
            est_data=est_data[pd.notnull(est_data["EST_EPS_DILUTED"])]
            est_data=est_data.groupby("ANALYST_NAME").apply(lambda x:x.iloc[-1])#同一分析师多篇研报只取最后一篇

        #如果过去180天没有研报，则返回空的DataFrame
        if len(est_data)==0:
            return pd.DataFrame(index=[TODAY_DATE],columns=["con_est_eps"],data=[np.nan])

        con_est_data=con_est_calcu(est_data)
        single_stkperiod_result=pd.DataFrame(index=[TODAY_DATE],columns=["con_est_eps"])
        single_stkperiod_result.loc[TODAY_DATE,"con_est_eps"]=con_est_data

        return single_stkperiod_result

    df_est=df_est[df_est["REPORTING_PERIOD"]>=str(START_YEAR)+'1231']
    grouped=df_est.groupby(["sid","REPORTING_PERIOD"])
    con_est_eps_df=apply_parallel(grouped,Process_Single_StkPeriod)
    con_est_eps_df=con_est_eps_df.reset_index().\
                    rename(columns={"level_0":"sid","level_1":"REPORTING_PERIOD","level_2":"DataDate"})

    return con_est_eps_df
con_est_eps_df=Get_Con_Est_Eps(df_est)
print("finish getting con_est_eps\n\n")

##如果企业发布了业绩预报或者快报，则直接采取业绩预报或者快报的数据
def Replace_with_Notice(df):
    reporting_period = df["REPORTING_PERIOD"][df.index[0]]
    if reporting_period==str(START_YEAR)+'1231':
        sid=df["sid"][df.index[0]]
        totshare = df_now_tot_share.set_index('sid').loc[sid,'TOT_SHR']
        min_profit=df["S_PROFITNOTICE_NETPROFITMIN"][df.index[0]]
        max_profit=df["S_PROFITNOTICE_NETPROFITMAX"][df.index[0]]
        notice_eps=(min_profit+max_profit)/2/totshare
        df["con_est_eps"]=notice_eps
    return df

con_est_eps_df_merged=pd.merge(con_est_eps_df,df_ProfitNotice,on=["sid","REPORTING_PERIOD"],how="left")
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"],as_index=False).apply(Replace_with_Notice)


def Replace_with_Express(df):
    reporting_period=df["REPORTING_PERIOD"][df.index[0]]
    if reporting_period==str(START_YEAR)+'1231':
        sid=df["sid"][df.index[0]]
        totshare = df_now_tot_share.set_index('sid').loc[sid,'TOT_SHR']
        express_eps=df["NET_PROFIT_EXCL_MIN_INT_INC"][df.index[0]]/totshare/10000
        df["con_est_eps"]=express_eps
    return df

con_est_eps_df_merged=pd.merge(con_est_eps_df_merged,df_ProfitExpress,on=["sid","REPORTING_PERIOD"],how="left")
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"],as_index=False).apply(Replace_with_Express)
con_est_eps_df_merged=con_est_eps_df_merged[["sid","REPORTING_PERIOD","DataDate","con_est_eps"]]

#在企业发布正式年报之后，使用正式年报数据代替一致预期值
df_FA_EPS.reset_index(inplace=True)
def Replace_with_FA(df):
    reporting_period=df["REPORTING_PERIOD"][df.index[0]]
    if reporting_period==str(START_YEAR)+'1231' :
        sid=df["sid"][df.index[0]]
        ann_date=df["ANN_DT"][df.index[0]]
        totshare = df_now_tot_share.set_index('sid').loc[sid,'TOT_SHR']
        FA_eps=(df['S_FA_DEDUCTEDPROFIT']+df['S_FA_EXTRAORDINARY'])[df.index[0]]/totshare/10000

        df["con_est_eps"]=FA_eps
    return df

con_est_eps_df_merged=pd.merge(con_est_eps_df_merged,df_FA_EPS,on=["sid","REPORTING_PERIOD"],how="left")
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"],as_index=False).apply(Replace_with_FA)


#加入PE值和每天收盘价
con_est_eps_df_merged=con_est_eps_df_merged[["sid","REPORTING_PERIOD","DataDate","con_est_eps"]]
con_est_eps_df_merged_renewed=pd.merge(con_est_eps_df_merged,df_PETTM_2y_rollingmedian,on=["sid","DataDate"],how="left")
con_est_eps_df_merged_renewed=pd.merge(con_est_eps_df_merged_renewed,df_close,on=["sid","DataDate"],how="left")
print("finish adjusting con_est_eps\n\n")


def get_level(close,prices):
    prices = prices.copy()
    prices.sort()
    
    level=0
    benchmark=0
    for p in prices:
        if close<p:
            return level+(close-benchmark)/(p-benchmark)
        else:
            level = level+1
            benchmark = p
    if (prices[2]-prices[1])/prices[1]>0.1:
        return level+(close-benchmark)/(prices[2]-prices[1])
    else:
        return level+(close-benchmark)/(prices[1]*0.1)

"""
def get_level_from_rolling_data(arr):
    current_close=arr[251,3]
    current_estprices=arr[251,0:3]
    
    close_series=arr[:,3]
    bottom_series=arr[:,0]
    middle_series = arr[:,1]
    ##params adjusting
    if np.sum(close_series<(bottom_series+middle_series)/2)/len(close_series)>0.80:
        current_estprices_adj = current_close/current_estprices[1]*current_estprices
    else:
        current_estprices_adj = current_estprices
    return get_level(current_close,current_estprices),get_level(current_close,current_estprices_adj)

def get_rolling_data(t_date):
    trade_dates = get_n_previous_trade_dates(t_date,252)
    df_rolling_data =pd.DataFrame()
    for date in trade_dates:
        df_daily = pd.read_csv(r"/share/intern_share/stk_over_level/stock_over_level_{}.csv".format(date))
        df_daily["date"] = date
        df_rolling_data = pd.concat([df_rolling_data,df_daily])
    return df_rolling_data.sort_values(["date","sid"])        
"""
def Single_Stock_Over_Level(df):
    sid=df["sid"][df.index[0]]
    date=df["DataDate"][df.index[0]]

    result_df=pd.DataFrame(index=[date],columns=["over_level","p1","p2","p3","close_price","pe_rolling_mean"])

    prices=np.array([0,0,0,0],dtype=float)
    for i in range(4):
        try:
            reporting_period=str(int(date[0:4])+i-1)+"1231"
            con_est_eps=df[df["REPORTING_PERIOD"]==reporting_period]["con_est_eps"].values[0]
            pe_rolling_median=df[df["REPORTING_PERIOD"]==reporting_period]["pe_2y_rollingmedian"].values[0]
            prices[i]=con_est_eps*pe_rolling_median
        except IndexError as e:
            pass
    
    #填充第三年数据
    if prices[2]==0:
        prices[2]= prices[1]+(prices[1]-prices[0])
    
    #处理四年con_eps 非线性增长的情况
    if prices[0]>prices[1] and prices[1]<prices[2]:
        prices[0] = prices[1]-(prices[2]-prices[1])
    elif prices[1]>prices[2] and prices[0]<prices[2]:
        prices[1] = (prices[0]+prices[2])/2
    
    #填充第四年数据
    if prices[3] ==0:
        prices[3]=prices[2]+prices[2]-prices[1]
        
    #进行平滑（向未来一年进行滚动）
    month , day =( int(date[4:6]), int(date[6:8]) )
    day_num = (month-1)*30 +day
    prices_smooth = np.array([0,0,0],dtype=float)
    for i in range(3):
        prices_smooth[i] = (prices[i]*(360-day_num)+prices[i+1]*day_num)/360
    
    try:
        close=df["close"][df.index[0]]
        level=get_level(close,prices_smooth)
        result_df.loc[date,'over_level']=level
    except:
        result_df.loc[date,'over_level']=np.nan
    
    result_df.iloc[0,1:4] = prices_smooth
    result_df.iloc[0,4] = close
    result_df.iloc[0,5] = df["pe_2y_rollingmedian"].values[0]
    return result_df

All_stock_over_level=con_est_eps_df_merged_renewed.groupby("sid").apply(Single_Stock_Over_Level)
All_stock_over_level=All_stock_over_level.replace(np.inf,np.nan).dropna().reset_index(level=1,drop=True)
print("finish getting over_level\n\n")
All_stock_over_level.to_csv(r"/share/intern_share/stk_over_level/stock_over_level_{}.csv".format(TODAY_DATE),index_label="sid")
logging.info("{} finished".format(TODAY_DATE))


In [ ]:
temp_df = con_est_eps_df_merged[~pd.notnull(con_est_eps_df_merged["con_est_eps"])]
temp_df["REPORTING_PERIOD"].value_counts()
len(All_stock_over_level)

In [ ]:
All_stock_over_level[All_stock_over_level["p1"]>All_stock_over_level["p2"]]

In [ ]:
%run Price_over_Level_daily.py

In [ ]:
temp_df = pd.read_csv(r"/share/intern_share/stk_over_level/stock_over_level_{}.csv"\
                      .format(TODAY_DATE))
temp_df

In [ ]:
df_whs300=query_table("DailyBar",start_date="20200310",
                      end_date="20200310",fields=["w_cs500"])

df_whs300["sid"][df_whs300["w_cs500"]!=0]

In [ ]:
con_eps_hs300 = temp_df.set_index("sid").loc[df_whs300["sid"][df_whs300["w_cs500"]!=0]]
con_eps_hs300[pd.notnull(con_eps_hs300["over_level"])]

* 沪深300 覆盖率： 85%左右
* 中证500 覆盖率： 66%左右
* 全指 覆盖率： 40%左右

# 每只票文件 --> 每日文件

In [ ]:
import os
file_names = os.listdir(r"result/stk_est_eps/")
file_names.remove('.ipynb_checkpoints')

df_ls = []
for file_name in file_names:
    df_stk = pd.read_csv("result//stk_est_eps//"+file_name)
    df_stk["sid"] = file_name[8:17]
    df_stk = df_stk.set_index("sid").reset_index()
    df_stk = df_stk[df_stk["p0"]>0]
    df_ls.append(df_stk)
df_summary= pd.concat(df_ls)

In [ ]:
df_summary=df_summary[df_summary["DataDate"]>='2017-01-01']

df_summary.groupby("DataDate").apply(lambda x:x.sort_values("sid")\
                                     .to_csv(r"/share/intern_share/stk_over_level/stk_over_level_{}.csv".format(x["DataDate"].iloc[0]),index=False))

In [ ]:
df_summary

In [ ]:
df = pd.read_csv("//share//intern_share//stk_over_level//stk_over_level_2020-01-02.csv")
df